In [7]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

# Latent Dirichlet Allocation

LSI pozwala w pewnym sensie znajdowac kluczowe tematy w tekście i dla danego dokumentu określać najbliższy mu temat. Jest to podejście geometryczne. 

LDA jest podejściem probabilistycznym do modelowania tematów. Jest bardziej dokładny choć wolniejszy.

# Model LDA - Latent Dirichlet Allocation (ukryta alokacja Dirichleta)


Motywacja: przedstawienie tekstu jako mieszanki tematów.


Temat - rozkład prawdopodobieństwa na zbiorze słów.


Przykład:
*  <s>Mam</s> gorączkę <s>i</s> katar.
* Graliśmy <s>w</s> siatkówkę.
* Grając <s>w</s> piłkę, wzmacniamy organizm.


Ile "tematów" widzimy?
<br>

<br>

<br>

<br>

<br>

<br>

Intuicyjnie: dwa tematy: "sport" oraz "zdrowie".
* Pierwsze zdanie = 100% zdrowie
* Drugie zdanie = 100% sport
* Trzecie zdanie = 50% sport + 50% zdrowie

LDA to model probabilistyczny, który wykorzystuje dwie wartości prawdopodobieństwa: 

* P (słowo | tematy) 
* P (tematy | dokumenty) 

do stworzenia klastrów.


Proces budowy modelu LDA zakłada więc:
* identyfikację tematów reprezentowanych przez dokumenty korpusu,
* oszacowanie rozkładu prawdopodobieństw wystąpienia wyrazów dla każdego z tematów (stosowany jest rozkład Dirichleta),
* oszacowanie rozkładu prawdopodobieństw wystąpienia tematów w rozpatrywanych dokumentach (stosowany jest również rozkład Dirichleta).

Wykorzystując metodę identyfikacji słów kluczowych opartą na ukrytej alokacji Dirichleta można:
* dla każdego dokumentu obliczyć prawdopodobieństwo wystąpienia w nim każdego z tematów;
* dla każdego tematu obliczyć prawdopodobieństwo wystąpienia słów;
* obliczyć prawdopodobieństwa wystąpienia poszczególnych słów w dokumencie (jako iloczyny dwóch wyżej wymienionych prawdopodobieństw) 

# Generatywność

Mając prawdopodobieństwa $p_1 = P (słowo | tematy) $, $p_2 = P (tematy | dokumenty)$ możemy wygeneraować dokument:

- wybieramy (losujemy) prawdopodobieństwo przynależności dokumentu do tematu (z $p_1$) - dokuemnt zawsze należy do wielu tematów

- generujemy słowa w dokumencie - najpierw losujemy do jakiego tematu należy słowo, a potem generujemy słowo z tego tematu (z $p_2$)

# Uczenie

1. Idziemy przez wszystkie słowa i wszystkie dokumentu i losowo przyporządkowujemy je do tematów.
2. Iteracyjnie idzemy: bierzemy dokuemnt $d$  i słowo $w$ i przyporządkowujemy je do najlepiej pasującego tematu - maksymalizującego $P (słowo | tematy) 
* P (tematy | dokumenty) $
3. Po przejsciu przez wszystkie dokumenty aktualizujemy przydział słów do tematów i tematów do dokumentów i wracamy do punktu 2

# Zad

Na początek wczytujemy korpus z dysku. Użyjemy przykład stworzonego w poprzednim notebook-u.

In [8]:
# documents = ["Romeo and Juliet",
#          "Juliet: O happy dagger",
#          "Romeo died by dagger",
#          "'Live free or die', that’s the New-Hampshire’s motto",
#          "Did you know, New-Hampshire is in New-England"]

In [9]:
if (os.path.exists("tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
    corpus = corpora.MmCorpus('tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")



Used files generated from first tutorial


In [10]:
for d in corpus:
    print(d)

[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [11]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
print(scipy_csc_matrix.todense())

[[1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [12]:
print(dictionary.token2id)

{'juliet': 0, 'romeo': 1, 'dagger': 2, 'happy': 3, 'juliet:': 4, 'o': 5, 'by': 6, 'died': 7, "'live": 8, "die',": 9, 'free': 10, 'motto': 11, 'new-hampshire’s': 12, 'or': 13, 'that’s': 14, 'did': 15, 'is': 16, 'know,': 17, 'new-england': 18, 'new-hampshire': 19, 'you': 20}


# Budujemy model LDA
Budujemy model LDA i transformujemy dane

* **num_topics=2** oznacza ilość modelowanych tematów

In [13]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = model[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for d in corpus_lda:
    print(d)

[(0, 0.7573557), (1, 0.24264431)]
[(0, 0.12710506), (1, 0.87289494)]
[(0, 0.8710887), (1, 0.12891136)]
[(0, 0.07185894), (1, 0.92814106)]
[(0, 0.89790803), (1, 0.10209194)]


### Dla każdego dokumentu dostajemy prawdopodobieństwo przynależności dokumentu do danego tematu.

Możemy też zobaczyć z czego składają się tematy:

In [14]:
model.show_topics()

[(0,
  '0.086*"romeo" + 0.067*"dagger" + 0.067*"died" + 0.063*"by" + 0.058*"new-england" + 0.054*"did" + 0.054*"new-hampshire" + 0.053*"know," + 0.052*"you" + 0.050*"is"'),
 (1,
  '0.069*"dagger" + 0.057*"or" + 0.056*"die\'," + 0.055*"happy" + 0.055*"that’s" + 0.054*"\'live" + 0.054*"romeo" + 0.054*"motto" + 0.053*"new-hampshire’s" + 0.053*"free"')]

In [15]:
model.print_topics(2)

[(0,
  '0.086*"romeo" + 0.067*"dagger" + 0.067*"died" + 0.063*"by" + 0.058*"new-england" + 0.054*"did" + 0.054*"new-hampshire" + 0.053*"know," + 0.052*"you" + 0.050*"is"'),
 (1,
  '0.069*"dagger" + 0.057*"or" + 0.056*"die\'," + 0.055*"happy" + 0.055*"that’s" + 0.054*"\'live" + 0.054*"romeo" + 0.054*"motto" + 0.053*"new-hampshire’s" + 0.053*"free"')]

In [16]:
model.print_topics(num_topics=2, num_words=4)

[(0, '0.086*"romeo" + 0.067*"dagger" + 0.067*"died" + 0.063*"by"'),
 (1, '0.069*"dagger" + 0.057*"or" + 0.056*"die\'," + 0.055*"happy"')]

# Zad

Chcemy posortować słowa każdego tematu i wybrać 5 najważniejszych - co można powiedzieć o tematach?

Proszę zobaczyć na funkcje typu get_topics(), get_term_topics(...): https://radimrehurek.com/gensim/models/ldamodel.html

In [17]:
import numpy as np
topics = np.argsort(model.get_topics()[0,:])[::-1] #::-1 sortowanie w odwrotnej kolejności
for x in topics[:5]:
    print(dictionary[x])

romeo
dagger
died
by
new-england


In [18]:
model.get_topic_terms(topicid=0)

[(1, 0.086217955),
 (2, 0.06702697),
 (7, 0.06671357),
 (6, 0.062788315),
 (18, 0.05837199),
 (15, 0.054136433),
 (19, 0.053630922),
 (17, 0.05346427),
 (20, 0.051601455),
 (16, 0.049574506)]

In [19]:
topic2_terms = model.get_topic_terms(topicid=0)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

[('romeo', 0.086217955),
 ('dagger', 0.06702697),
 ('died', 0.06671357),
 ('by', 0.062788315),
 ('new-england', 0.05837199),
 ('did', 0.054136433),
 ('new-hampshire', 0.053630922),
 ('know,', 0.05346427),
 ('you', 0.051601455),
 ('is', 0.049574506)]

In [20]:
[
    (dictionary.get(i), j)
    for i,j in model.get_topic_terms(topicid=0)
]

[('romeo', 0.086217955),
 ('dagger', 0.06702697),
 ('died', 0.06671357),
 ('by', 0.062788315),
 ('new-england', 0.05837199),
 ('did', 0.054136433),
 ('new-hampshire', 0.053630922),
 ('know,', 0.05346427),
 ('you', 0.051601455),
 ('is', 0.049574506)]

# Zad 

Proszę posortować zdania najbardziej pasujące do danego tematu. 

In [21]:
import gensim

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    
#trzeba by wypisać raczej zdania niż ich reprezentacje bag-of-words, ale tu nie mam dostepu do tekstu

[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(0, 1.0), (1, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]


In [22]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(corpus[x])
    

[(8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0)]
[(2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]
[(0, 1.0), (1, 1.0)]
[(1, 1.0), (2, 1.0), (6, 1.0), (7, 1.0)]
[(15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0)]


In [23]:
documents = ["Romeo and Juliet",
         "Juliet: O happy dagger",
         "Romeo died by dagger",
         "'Live free or die', that’s the New-Hampshire’s motto",
         "Did you know, New-Hampshire is in New-England"]

top_inex = 0

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort( numpy_corpus[top_inex,:] )[::-1]
for x in docs[:5]:
    print(documents[x])
    

Did you know, New-Hampshire is in New-England
Romeo died by dagger
Romeo and Juliet
Juliet: O happy dagger
'Live free or die', that’s the New-Hampshire’s motto


In [24]:
top_inex = 1

numpy_corpus = gensim.matutils.corpus2dense(corpus_lda, num_terms=2)
docs = np.argsort(numpy_corpus[top_inex,:])[::-1]
for x in docs[:5]:
    print(documents[x])
    

'Live free or die', that’s the New-Hampshire’s motto
Juliet: O happy dagger
Romeo and Juliet
Romeo died by dagger
Did you know, New-Hampshire is in New-England


# Zad. 
Sprawdzić do jakiego tematu pasuje nowy dokument i jakie są mu najbliższe

In [25]:
doc = "died dagger"

In [26]:
doc_rep = dictionary.doc2bow(doc.split(' '))
# print(doc_rep)
doc_assignments = model[doc_rep]
print(doc_assignments)

[(0, 0.7674487), (1, 0.23255134)]


In [27]:
index = similarities.MatrixSimilarity(corpus_lda)

sims = index[doc_assignments]
print(list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.9998754), (1, 0.4249646), (2, 0.98917943), (3, 0.3630086), (4, 0.9836546)]
[(0, 0.9998754), (2, 0.98917943), (4, 0.9836546), (1, 0.4249646), (3, 0.3630086)]


# Wizualizacja modelu LDA:

pyLDAvis

http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb


* Czerwone słupki reprezentują częstotliwość słów w danym temacie (proporcjonalnie do $P (słowo \ | \ tematy) $), 
* Niebieskie słupki reprezentują częstotliwość tematów w całym korpusie (proporcjonalnie do $P(tematy \ | \ dokumenty)$. 

Zmień wartość $\lambda$, aby dostosować rankingi słów: 
 * małe wartości $\lambda$ (blisko 0) podkreślają potencjalnie rzadkie, ale ekskluzywne słowa dla wybranego tematu
 * duże wartości $\lambda$ (blisko 1) podkreślają częste, ale niekoniecznie ekskluzywne słowa dla wybranego tematu. 
 
W http://www.kennyshirley.com/LDAvis/ sugeruje się żeby ustawiać $\lambda$ w pobliżu 0.6. Podobno pomaga to użytkownikom w interpretacji tematu.

In [28]:
import pyLDAvis.gensim

In [29]:
pyLDAvis.enable_notebook()

In [30]:
# pyLDAvis.gensim.prepare??

In [31]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

C:\Users\przem\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.011679  0.0       1        1  50.442539
0     -0.011679  0.0       2        1  49.557457, topic_info=     Category      Freq             Term     Total  loglift  logprob
term                                                                
7     Default  1.000000             died  1.000000  21.0000  21.0000
6     Default  1.000000               by  1.000000  20.0000  20.0000
1     Default  1.000000            romeo  1.000000  19.0000  19.0000
13    Default  1.000000               or  1.000000  18.0000  18.0000
18    Default  1.000000      new-england  1.000000  17.0000  17.0000
9     Default  1.000000            die',  1.000000  16.0000  16.0000
3     Default  1.000000            happy  1.000000  15.0000  15.0000
14    Default  1.000000           that’s  1.000000  14.0000  14.0000
8     Default  1.000000            'live  1.000000  13.0000  13.0000
11    Default  1.000000            motto  1.000000  12.0000  12.0000
12    Default  1.000000  new-hampshire’s  1.000000  11.0000  11.0000
10    Default  1.000000             free  1.000000  10.0000  10.0000
15    Default  1.000000              did  1.000000   9.0000   9.0000
4     Default  1.000000          juliet:  1.000000   8.0000   8.0000
19    Default  1.000000    new-hampshire  1.000000   7.0000   7.0000
17    Default  1.000000            know,  1.000000   6.0000   6.0000
20    Default  1.000000              you  1.000000   5.0000   5.0000
5     Default  1.000000                o  1.000000   4.0000   4.0000
16    Default  1.000000               is  1.000000   3.0000   3.0000
0     Default  1.000000           juliet  1.000000   2.0000   2.0000
2     Default  1.000000           dagger  1.000000   1.0000   1.0000
13     Topic1  0.660716               or  1.014253   0.2558  -2.8656
9      Topic1  0.645558            die',  1.017799   0.2291  -2.8888
3      Topic1  0.638096            happy  1.019545   0.2157  -2.9004
14     Topic1  0.637694           that’s  1.019639   0.2150  -2.9011
8      Topic1  0.627744            'live  1.021967   0.1970  -2.9168
11     Topic1  0.621835            motto  1.023349   0.1862  -2.9262
12     Topic1  0.619716  new-hampshire’s  1.023845   0.1823  -2.9297
10     Topic1  0.615300             free  1.024878   0.1741  -2.9368
4      Topic1  0.611833          juliet:  1.025689   0.1677  -2.9425
...       ...       ...              ...       ...      ...      ...
16     Topic1  0.489298               is  1.054359  -0.0834  -3.1659
20     Topic1  0.470574              you  1.058739  -0.1265  -3.2050
17     Topic1  0.453368            know,  1.062765  -0.1676  -3.2422
19     Topic1  0.451828    new-hampshire  1.063125  -0.1713  -3.2456
15     Topic1  0.447159              did  1.064218  -0.1827  -3.2560
1      Topic1  0.624428            romeo  1.607159  -0.2611  -2.9221
18     Topic1  0.408034      new-england  1.073371  -0.2829  -3.3476
6      Topic1  0.367240               by  1.082915  -0.3971  -3.4529
7      Topic1  0.330983             died  1.091399  -0.5088  -3.5568
7      Topic2  0.760416             died  1.091399   0.3407  -2.7073
6      Topic2  0.715675               by  1.082915   0.2879  -2.7680
18     Topic2  0.665337      new-england  1.073371   0.2238  -2.8409
1      Topic2  0.982731            romeo  1.607159   0.2101  -2.4509
15     Topic2  0.617059              did  1.064218   0.1570  -2.9162
19     Topic2  0.611297    new-hampshire  1.063125   0.1487  -2.9256
17     Topic2  0.609397            know,  1.062765   0.1459  -2.9287
20     Topic2  0.588165              you  1.058739   0.1142  -2.9642
16     Topic2  0.565061               is  1.054359   0.0783  -3.0043
0      Topic2  0.563757           juliet  1.054111   0.0762  -3.0066
2      Topic2  0.763988           dagger  1.565685  -0.0155  -2.7027
5      Topic2  0.442862                o  1.031189  -0.1432  -3.2480
4      Topic2  0.413857          ju